In [25]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Module 2: Create Landsat and Sentinel 2 Annual Composites

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/redcastle-resources/lcms-training/blob/main/2-Composites.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/redcastle-resources/lcms-training/blob/main/2-Composites.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/redcastle-resources/lcms-training/blob/main/2-Composites.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>
<br/><br/><br/>

## Overview


This notebook introduces the compositing methods used by the USDA Forest Service Landscape Change Monitoring System (LCMS). It then produces composites for a small LCMS study area. The composites will be used to generate time series of data that the LCMS modeling algorithms will use to predict areas of land use and land cover change, in the following modules. 

**Learn more about [LCMS](https://apps.fs.usda.gov/lcms-viewer/home.html).**


All composites are created and visualized using the `geeViz` Python package. `geeViz` provides command-line tools for processing, analyzing, and visualizing Google Earth Engine (GEE) code. 

**Learn more about [geeViz](https://github.com/gee-community/geeViz/tree/master).**


### Objective

##### This tutorial uses the following Google Cloud services:

- `Google Earth Engine`
- `Google Cloud Storage`

##### The steps performed include:

- Install necessary packages
- Test various parameters to create composites
    - Run with default parameters
    - Include Landsat 7 in image composites
    - Set cloud masking settings
    - Set cloud shadow masking settings
- Create and export composite to asset

# Before you begin

### Set your current URL under `workbench_url`
This gives the Map Viewer a url in which to host the viewer we will be generating. 
* This will be in your URL/search bar at the top of the browser window you are currently in
* It will look something like `https://1234567890122-dot-us-west3.notebooks.googleusercontent.com/` (See the image below)

![workspace url](img/workspace-url.png)

### Set a folder to use for all exports under `export_path_root` 
This folder should be an assets folder in an existing GEE project. 
* It will be something like `projects/projectID/assets/someFolder`
* This folder does not have to already exist. If it does not exist, it will be created

In [2]:
workbench_url = 'https://3b40cb8a2076f1b6-dot-us-west3.notebooks.googleusercontent.com/'
export_path_root  = 'projects/rcr-gee/assets/lcms-training'

print('Done')

Done


# Installation
First, install necessary Python packages. Uncomment the first line to upgrade geeViz if necessary.

In [3]:
#Module imports
#!python -m pip install geeViz --upgrade
try:
    import geeViz.getImagesLib as getImagesLib
except:
    !python -m pip install geeViz
    import geeViz.getImagesLib as getImagesLib

import geeViz.assetManagerLib as aml
import geeViz.taskManagerLib as tml
# from IPython.display import IFrame,display, HTML
ee = getImagesLib.ee
Map = getImagesLib.Map

print('Done')

Initializing GEE
Successfully initialized
geeViz package folder: /opt/conda/lib/python3.10/site-packages/geeViz
Done


## Set up your work environment

Create a folder in your export path where you will export the composites. In addition, create a blank image collection where your composites will live.

Currently, when running within Colab, geeView uses a different project to authenticate through, so you may need to make your asset public to view from within Colab.

In [4]:
# Create folder 
export_composite_collection = f'{export_path_root}/lcms-training_module-2_composites'

# Create a collection
aml.create_asset(export_composite_collection,asset_type = ee.data.ASSET_TYPE_IMAGE_COLL)

# Make it public
aml.updateACL(export_composite_collection,writers = [],all_users_can_read = True,readers = [])


# aml.batchCopy('projects/rcr-gee/assets/composites-lcms-training-module-1',export_composite_collection,outType = 'imageCollection')
# # aml.batchDelete('projects/rcr-gee/assets/composites-lcms-training-module-1')
# print('Done')

Found the following sub directories:  ['lcms-training', 'lcms-training_module-2_composites']
Will attempt to create them if they do not exist
Asset projects/rcr-gee/assets/lcms-training already exists
Asset projects/rcr-gee/assets/lcms-training/lcms-training_module-2_composites already exists
Updating permissions for:  projects/rcr-gee/assets/lcms-training/lcms-training_module-2_composites


## Example 1: Default parameters

Over the course of the examples in this module, you'll become more familiar with the parameters used to create image composites. The main function you will use is `getLandsatAndSentinel2HybridWrapper`. 

For more documentation of the parameters in the function, refer to the [getCombinedLandsatSentinel2Wrapper](https://github.com/gee-community/geeViz/blob/master/examples/getCombinedLandsatSentinel2Wrapper.py) example in the geeViz documentation.

You can view the [source code](https://github.com/gee-community/geeViz/blob/bd75d38c2043fc3af795c9c35ae769c71c00f56a/getImagesLib.py#L3280) for this function on the geeViz GitHub.

### Set up map
The first thing you'll do is set up the map viewer where you'll inspect the outputs you generate. You won't view the map until later. 

In [5]:
# Set proxy url
Map.proxy_url = workbench_url

#First clear the map in case it has been populated with layers/commands earlier
Map.clearMap()

### Define parameters

#### The first parameters you define will set the **area** and **time range** over which you want to compute annual composites.

* ```studyArea``` - a featureCollection, feature or geometry to specify the bounds of our area of interest

* ```startJulian``` - the first julian date to include in annual composites, used to specify a particular season if desired
* ```endJulian``` - the last julian date to include in annual composites, used to specify a particular season if desired

    This supports wrapping for tropics and southern hemisphere. If you are using wrapping and the majority of the days occur in the second year, the system:time_start will default to June 1 of that year. Otherwise, all system:time_starts will default to June 1 of the given year.

* ```startYear``` - the first year for which to compute annual composites
* ```endYear``` - the last year for which to compute annual composites

    You should provide more than a 3 year span should be provided for time series methods to work well. If you are providing pre-computed stats for cloud masking using cloudScore and TDOM, this does not matter. 

In [6]:
# Define user parameters:

# Specify study area
studyArea = ee.FeatureCollection('projects/lcms-292214/assets/R8/PR_USVI/Ancillary/prusvi_boundary_buff2mile')#.geometry().bounds(50,'EPSG:5070')
## Raw geometry for polygon provided below
# studyArea = ee.Geometry.Polygon(
#         [[[-66.29465453845745, 18.491553939984392],
#           [-66.29465453845745, 18.144770192006572],
#           [-65.58054321033245, 18.144770192006572],
#           [-65.58054321033245, 18.491553939984392]]], None, False)

# Update the startJulian and endJulian 

startJulian = 152
endJulian = 151

# Specify start and end years for all analyses

startYear = 2009
endYear = 2011

### Build composites 
Using the parameters you specified above, you will create a series of image composites. By default, the script prints out all parameters that are used in acquiring the Landsat and Sentinel-2 imagery. You will notice that there are many parameters that are set to their defaults! Over the next few examples, we'll adjust a few more of these parameters. 

You can review all of the parameters, and their defaults, in the [getCombinedLandsatSentinel2Wrapper](https://github.com/gee-community/geeViz/blob/master/examples/getCombinedLandsatSentinel2Wrapper.py).

Set verbose = FALSE (or whatever this is!!!) to make it quieter

In [8]:
#Call on master wrapper function to get Landat scenes and composites
lsAndTs = getImagesLib.getLandsatAndSentinel2HybridWrapper(studyArea.geometry().bounds(50,'EPSG:5070'),
                                                           startYear,
                                                           endYear,
                                                           startJulian,
                                                           endJulian)

#Separate into scenes and composites for subsequent analysis
processedScenes = lsAndTs['processedScenes']
processedComposites = lsAndTs['processedComposites']

Get Processed Landsat and Sentinel2 Scenes: 
Start date: Jun 01 2009 , End date: May 30 2012
Get Processed Landsat: 
Start date: Jun 01 2009 , End date: May 30 2012
Only including SLC On Landsat 7
Applying Fmask Cloud Mask
Applying TDOM Shadow Mask
Computing irMean for TDOM
Computing irStdDev for TDOM
Applying Fmask Shadow Mask
Get Processed Sentinel2: 
Start date: Jun 01 2009 , End date: May 30 2012
Using S2 Collection: COPERNICUS/S2_HARMONIZED
Joining pre-computed cloud probabilities from: COPERNICUS/S2_CLOUD_PROBABILITY
Setting to aggregate instead of resample 
Converting S2 data to daily mosaics
Apply Cloud Probability
Applying TDOM
Computing irMean for TDOM
Computing irStdDev for TDOM
Running Chastain et al 2019 harmonization


### Add to Map
The `geeView` module of the `geeViz` package provides functions for creating an interactive map viewer outside of the Google Earth Engine Code Editor and Playground. Generally, `Map` functions in `geeView` follow the same syntax as javascript Earth Engine commands in the playground. 

For more information and examples of geeView capabilities, visit the [geeView example](https://github.com/gee-community/geeViz/blob/master/examples/geeViewExample.py) in the geeViz github. 

The interactive map viewer will appear below.

In [9]:
Map.clearMap()

Map.addTimeLapse(processedComposites,getImagesLib.vizParamsFalse,'Default Params {}-{}'.format(startJulian,endJulian),'True')
Map.addLayer(studyArea,{},'Study Area')
Map.centerObject(studyArea)
Map.turnOnInspector()
Map.view()

Adding layer: Default Params 152-151
Adding layer: Study Area
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Starting local web server at: http://localhost:8001/geeView/
HTTP server command: "/opt/conda/bin/python" -m http.server  8001
Done
cwd /home/jupyter/lcms-training


Please enter current URL Workbench Notebook is running from (e.g. https://code-dot-region.notebooks.googleusercontent.com/):  https://3b40cb8a2076f1b6-dot-us-west3.notebooks.googleusercontent.com/


Workbench Proxy URL: https://3b40cb8a2076f1b6-dot-us-west3.notebooks.googleusercontent.com/proxy/8001/geeView/?accessToken=None


127.0.0.1 - - [09/Aug/2023 20:35:20] "GET /geeView/?accessToken=None HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 20:35:20] "GET /geeView/css/style.min.css HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 20:35:20] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 20:35:20] "GET /geeView/js/lcms-viewer.min.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 20:35:20] "GET /geeView/js/load.min.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 20:35:20] "GET /geeView/js/gena-gee-palettes.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 20:35:21] "GET /geeView/images/GEE_logo_transparent.png HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 20:35:21] "GET /geeView/images/layer_icon.png HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 20:35:21] "GET /geeView/images/logos_usda-fs.svg HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 20:35:21] "GET /geeView/images/usfslogo.png HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 20:35:21] "GET /geeView/images/GEE.png HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 20:35:21]

#### Inspect the outputs

Look at the map that appeared above. You may need to click the button next to the layer titled "Default Params" in order to view the layer. Click the play button 
![play button](img/play-button.png) to show a time lapse of all of the annual composites created.  

**It is clear the default parameters do not work very well in this area.**
Few, if any of the years displayed have complete images for the annual composite. Data are missing for large swaths of the immage, and there are cloud artifacts clearly visible.

Though the results panel above indicates operations that were applied to create the composites, it doesn't tell you the full list of parameters used to create the composite. You can access the parameters that were used through the properties of the returned collection.

In [ ]:
#Print parameters used
#Parameters are stored in properties of returnd collection
print(processedComposites.toDictionary().getInfo())
print(processedScenes.toDictionary().getInfo())



## Example 2: Include Landsat 7
Since there are not that many images available in this area for these years, let's try adding Landsat 7. Our default is to exclude Landsat 7 from image composites because of the [Landsat 7 scan line correction failure](https://www.usgs.gov/landsat-missions/landsat-7?qt-science_support_page_related_con=0#qt-science_support_page_related_con).

### Add a new parameter

To include Landsat 7, you will add a new parameter to the wrapper function. 

``` includeSLCoffL7 ``` - default: **False** 

Run the code block below to add the new parameter.

In [7]:
#add LS 7 parameter
includeSLCOffL7 = True

### Call on wrapper function again, including new parameter

Now, run the same wrapper function again. Note that all of the parameters are the same, with the addition of your new `includeSLCOffL7` parameter.

In [12]:
#Call on master wrapper function to get Landsat scenes and composites
lsAndTs = getImagesLib.getLandsatAndSentinel2HybridWrapper(studyArea.geometry().bounds(50,'EPSG:5070'),
                                                           startYear,
                                                           endYear,
                                                           startJulian,
                                                           endJulian,
                                                           includeSLCOffL7=includeSLCOffL7)


#Separate into scenes and composites for subsequent analysis
processedScenes = lsAndTs['processedScenes']
processedComposites = lsAndTs['processedComposites']

Get Processed Landsat and Sentinel2 Scenes: 
Start date: Jun 01 2009 , End date: May 30 2012
Get Processed Landsat: 
Start date: Jun 01 2009 , End date: May 30 2012
Including All Landsat 7
Applying Fmask Cloud Mask
Applying TDOM Shadow Mask
Computing irMean for TDOM
Computing irStdDev for TDOM
Applying Fmask Shadow Mask
Get Processed Sentinel2: 
Start date: Jun 01 2009 , End date: May 30 2012
Using S2 Collection: COPERNICUS/S2_HARMONIZED
Joining pre-computed cloud probabilities from: COPERNICUS/S2_CLOUD_PROBABILITY
Setting to aggregate instead of resample 
Converting S2 data to daily mosaics
Apply Cloud Probability
Applying TDOM
Computing irMean for TDOM
Computing irStdDev for TDOM
Running Chastain et al 2019 harmonization


### Add to map

Run the codeblock below to add the composites you created to the map again to inspect your results.

In [13]:
#Turn off layers from previous iteration
Map.turnOffAllLayers()

# Map.addLayer(processedComposites.select(['NDVI','NBR']),{'addToLegend':'false'},'Time Series (NBR and NDVI)',False)
# for year in range(startYear,endYear + 1 ):
    #  t = processedComposites.filter(ee.Filter.calendarRange(year,year,'year')).mosaic()
Map.addTimeLapse(processedComposites,getImagesLib.vizParamsFalse,'L7 added {}-{}'.format(startJulian,endJulian),True)


Map.view()

Adding layer: L7 added 152-151
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:8001/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://3b40cb8a2076f1b6-dot-us-west3.notebooks.googleusercontent.com/proxy/8001/geeView/?accessToken=None


127.0.0.1 - - [09/Aug/2023 20:53:55] "GET /geeView/?accessToken=None HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 20:53:55] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 20:53:55] "GET /geeView/js/load.min.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 20:53:55] "GET /geeView/js/gena-gee-palettes.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 20:53:55] "GET /geeView/js/lcms-viewer.min.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 20:53:55] "GET /geeView/css/style.min.css HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 20:53:55] "GET /geeView/images/GEE_logo_transparent.png HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 20:53:55] "GET /geeView/images/layer_icon.png HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 20:53:55] "GET /geeView/images/menu-hamburger_ffffff.svg HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 20:53:55] "GET /geeView/images/logos_usda-fs.svg HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 20:53:55] "GET /geeView/images/usfslogo.png HTTP/1.1" 200 -
127.0.0.1 - - [09/

### Inspect the outputs
Turn on the layers in the map viewer if necessary. 

Toggle back and forth between the Default Params and the new composites with Landsat 7 added. Zoom in to view the composites in more detail.

You'll notice this helps fill in the holes, but introduces many cloud-related artifacts.

## Example 3: Improve Cloud masking 
Next you'll improve the cloud masking. [Fmask](https://www.sciencedirect.com/science/article/abs/pii/S0034425714005069) is used by default, but this algorithm misses some clouds.

We'll try the adding in the [cloudScore](https://developers.google.com/earth-engine/guides/landsat#simple-cloud-score) method to mask clouds in Landsat.

We do this by setting the ```applyCloudScoreLandsat``` parameter

### Add cloud masking parameter

Run the code block below to add the `applyCloudScoreLandsat` parameter.

In [8]:
# set cloudScore
applyCloudScoreLandsat = True

### Run master wrapper function

Keeping all other parameters the same, run the `getLandsatAndSentinel2HybridWrapper` function again. This time, add in the new parameter.

In [26]:
#Call on master wrapper function to get Landsat scenes and composites
lsAndTs = getImagesLib.getLandsatAndSentinel2HybridWrapper(studyArea.geometry().bounds(50,'EPSG:5070'),
                                                           startYear,
                                                           endYear,
                                                           startJulian,
                                                           endJulian,
                                                           includeSLCOffL7=includeSLCOffL7,
                                                           applyCloudScoreLandsat=applyCloudScoreLandsat)


#Separate into scenes and composites for subsequent analysis
processedScenes = lsAndTs['processedScenes']
processedComposites = lsAndTs['processedComposites']

### Add to Map

Run the code block below to add the new layers to the map.

In [17]:
#Turn off layers from previous iteration
Map.turnOffAllLayers()


Map.addTimeLapse(processedComposites,getImagesLib.vizParamsFalse,'L7 and CloudScore added {}-{}'.format(startJulian,endJulian),True)


Map.view()

Adding layer: L7 and CloudScore added 152-151
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:8001/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://3b40cb8a2076f1b6-dot-us-west3.notebooks.googleusercontent.com/proxy/8001/geeView/?accessToken=None


127.0.0.1 - - [09/Aug/2023 21:52:30] "GET /geeView/?accessToken=None HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 21:52:30] "GET /geeView/js/gena-gee-palettes.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 21:52:30] "GET /geeView/css/style.min.css HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 21:52:30] "GET /geeView/js/lcms-viewer.min.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 21:52:30] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 21:52:30] "GET /geeView/js/load.min.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 21:52:31] "GET /geeView/images/GEE_logo_transparent.png HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 21:52:31] "GET /geeView/images/GEE.png HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 21:52:31] "GET /geeView/images/logos_usda-fs.svg HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 21:52:31] "GET /geeView/images/usdalogo.png HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 21:52:31] "GET /geeView/images/layer_icon.png HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 21:52:31]

### Inspect
Toggle on the layers in the viewer above. Inspect the **L7 and cloudScore** output. Notice that these layers look much better and have fewer cloud masking artifacts than the previous layers. 

However, you can still see some dark ares. These are likely due to cloud shadows that were not masked out-- the Fmask cloud shadow mask typically does not mask out all cloud shadows. 

To mask cloud shadow, we use a method called the [Temporal Dark Outlier Mask (TDOM)](https://www.sciencedirect.com/science/article/pii/S0034425718305212) that works well to mask cloud shadows. 



## Example 4: Use Temporal Dark Outlier Mask (TDOM) to mask cloud shadows
The Temporal Dark Outlier Mask (TDOM) method of masking cloud shadows calculates the median and standard deviation of the NIR and SWIR bands over a pixel in an image stack. The algorithm applies thresholds to determine which pixels are likely to be cloud shadows.

Find [details of the TDOM method](https://github.com/gee-community/geeViz/blob/bd75d38c2043fc3af795c9c35ae769c71c00f56a/getImagesLib.py#L1141) in the geeViz library.

### Set TDOM parameter

``` applyTDOM ``` - default = **False** determines whether or not TDOM will be used to mask cloud shadows.

Run the code block below to set the TDOM parameter.

In [9]:
#Set TDOM to true
applyTDOM = True

### Run master wrapper function

In [13]:
#Call on master wrapper function to get Landat scenes and composites
#In order to identify dark outliers, we will extend the dates by 6 years to get a larger sample
lsAndTs = getImagesLib.getLandsatAndSentinel2HybridWrapper(studyArea.geometry().bounds(50,'EPSG:5070'),
                                                           startYear,
                                                           endYear,
                                                           startJulian,
                                                           endJulian,
                                                           includeSLCOffL7=includeSLCOffL7,
                                                           applyCloudScoreLandsat=applyCloudScoreLandsat,
                                                           applyTDOMLandsat =applyTDOM,
                                                           applyTDOMSentinel2 = applyTDOM)


#Separate into scenes and composites for subsequent analysis
processedScenes = lsAndTs['processedScenes']
processedComposites = lsAndTs['processedComposites']

Get Processed Landsat and Sentinel2 Scenes: 
Start date: Jun 01 2009 , End date: May 30 2012
Get Processed Landsat: 
Start date: Jun 01 2009 , End date: May 30 2012
Including All Landsat 7
Applying Cloud Score
Computing cloudScore offset
Applying Fmask Cloud Mask
Applying TDOM Shadow Mask
Computing irMean for TDOM
Computing irStdDev for TDOM
Applying Fmask Shadow Mask
Get Processed Sentinel2: 
Start date: Jun 01 2009 , End date: May 30 2012
Using S2 Collection: COPERNICUS/S2_HARMONIZED
Joining pre-computed cloud probabilities from: COPERNICUS/S2_CLOUD_PROBABILITY
Setting to aggregate instead of resample 
Converting S2 data to daily mosaics
Apply Cloud Probability
Applying TDOM
Computing irMean for TDOM
Computing irStdDev for TDOM
Running Chastain et al 2019 harmonization


### Add to map

Run the code block below to add the new layers to the map.

In [14]:
#Turn off layers from previous iteration
Map.turnOffAllLayers()

Map.addTimeLapse(processedComposites,getImagesLib.vizParamsFalse,'CloudScore and TDOM added {}-{}'.format(startJulian,endJulian),True)

Map.view()

Adding layer: CloudScore and TDOM added 152-151
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Starting local web server at: http://localhost:8001/geeView/
HTTP server command: "/opt/conda/bin/python" -m http.server  8001
Done
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://3b40cb8a2076f1b6-dot-us-west3.notebooks.googleusercontent.com/proxy/8001/geeView/?accessToken=None


127.0.0.1 - - [09/Aug/2023 22:16:12] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -


### Inspect
Toggle on the new layer. You'll notice this cleans up the cloud masking a lot. 

### TDOM is computationally intensive
However, while TDOM is a great cloud shadow masking method, it is a bit computationally intensive since it computes the mean and standard deviation over a large stack of data.

In order to avoid re-computing the stats, we store pre-computed stats. For the LCMS project, we have precomputed stats for CONUS, AK, HI, PuertoRico, and the USVI. 
- (do we want to point people to these stats? )

It is optional that you use pre-computed stats. If none are provided, TDOM stats will be computed on-the-fly. 

You can create and store precomputed TDOM stats using the following code chunk: 

```
{
  this is an example set of code to show people how to compute TDOM stats
  we'll fill this in later!!
}
```
### Inspect raw, precomputed TDOM stats

Run the chuck of code below to load the precomputed TDOM stats and examine them on the map. 

In [15]:
# clear map
Map.clearMap()

# load precomputed tdom stats
landsat_tdom_stats = ee.Image('projects/lcms-tcc-shared/assets/CS-TDOM-Stats/PR-USVI/TDOM_stats/Landsat_TDOM_Stats_1984_2021')\
                    .select(['Landsat_nir_.*','Landsat_swir1_.*'])\
                    .divide(10000)
s2_tdom_stats = ee.Image('projects/lcms-tcc-shared/assets/CS-TDOM-Stats/PR-USVI/TDOM_stats/Sentinel2_TDOM_Stats_2015_2021')\
                    .select(['Sentinel2_nir_.*','Sentinel2_swir1_.*'])\
                    .divide(10000)
Map.addLayer(landsat_tdom_stats,{'min':0.15,'max':0.35,'bands':'Landsat_nir_mean','palette':'222,080'},'Landsat TDOM Stats')
Map.addLayer(s2_tdom_stats,{'min':0.15,'max':0.35,'bands':'Sentinel2_nir_mean','palette':'222,080'},'Sentinel 2 TDOM Stats')

Map.turnOnInspector()
Map.view()


Adding layer: Landsat TDOM Stats
Adding layer: Sentinel 2 TDOM Stats
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:8001/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://3b40cb8a2076f1b6-dot-us-west3.notebooks.googleusercontent.com/proxy/8001/geeView/?accessToken=None


127.0.0.1 - - [09/Aug/2023 22:19:14] "GET /geeView/?accessToken=None HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 22:19:14] "GET /geeView/js/gena-gee-palettes.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 22:19:14] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 22:19:14] "GET /geeView/css/style.min.css HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 22:19:14] "GET /geeView/js/load.min.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 22:19:14] "GET /geeView/js/lcms-viewer.min.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 22:19:14] "GET /geeView/images/GEE.png HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 22:19:14] "GET /geeView/images/GEE_logo_transparent.png HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 22:19:14] "GET /geeView/images/menu-hamburger_ffffff.svg HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 22:19:14] "GET /geeView/images/usfslogo.png HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 22:19:14] "GET /geeView/images/layer_icon.png HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 2

#### Example 4 - Prepare final composites and export
Now that you've gone through some of the parameters that can be changed for tge composites, we will use both Landsat and Sentinel2 to get the best possible composites. 

Here, we're manually setting all parameters for the `getLandatAndSentinel2HybridWrapper`. 


Review the [documentation](https://github.com/redcastle-resources/geeViz/blob/bd75d38c2043fc3af795c9c35ae769c71c00f56a/getImagesLib.py#L3280) to recall the default values for each of these parameters.

Note in the function below where the parameters are different from the defaults. 

Run the code block below to prepare and export composites. 

In [12]:
# Run get images, set parameters manually, and export
lsAndTs = getImagesLib.getLandsatAndSentinel2HybridWrapper(\
  
                                                          # study area and time                                                          
                                                          studyArea = studyArea,
                                                          startYear = 1984,
                                                          endYear = 2022,
                                                          startJulian = 152,
                                                          endJulian = 151,
                                                          timebuffer =  0,
                                                          weights =  [1],
  
                                                           # composite methods and input imagery
                                                          compositingMethod = 'medoid',
                                                          toaOrSR = 'TOA',
                                                          includeSLCOffL7 = True,
                                                          defringeL5 = True,
                                                          landsatCollectionVersion = 'C2',
                                                           
                                                           # cloud and shadow masking
                                                          applyQABand = False,
                                                          applyCloudProbability = True,
                                                          applyShadowShift = False,
                                                          applyCloudScoreLandsat = True,
                                                          applyCloudScoreSentinel2 = False,
                                                          applyTDOMLandsat = True,
                                                          applyTDOMSentinel2 = True,
                                                          applyFmaskCloudMask = True,
                                                          applyFmaskCloudShadowMask = True,
                                                          applyFmaskSnowMask = False,
                                                          cloudHeights = ee.List.sequence(500,10000,500),
                                                          cloudScoreThresh = 10,
                                                          performCloudScoreOffset = False,
                                                          cloudScorePctl = 10,
                                                          zScoreThresh = -1,
                                                          shadowSumThresh = 0.35,
                                                          contractPixels = 1.5,
                                                          dilatePixels = 3.5,
                                                          shadowSumBands = ['nir','swir1'],
                                                          cloudProbThresh = 40,
                                                           
                                                           #image processing
                                                          landsatResampleMethod = 'bicubic',
                                                          sentinel2ResampleMethod = 'bicubic',
                                                          convertToDailyMosaics = True,
                                                          runChastainHarmonization = False, # Set to True in order to perform regression harmonization between Landsat and Sentinel 2
                                                          correctIllumination = False,
                                                          correctScale = 250,
                                                           
                                                           # export parameters
                                                          exportComposites = True,
                                                          outputName = 'Landsat-Sentinel2-Hybrid',
                                                          exportPathRoot = export_composite_collection,
                                                          crs = getImagesLib.common_projections['NLCD_CONUS']['crs'],
                                                          transform = getImagesLib.common_projections['NLCD_CONUS']['transform'],
                                                          scale = None,
                                                           overwrite = False,
                                                           
                                                           # precomputed TDOM and cloudscore stats
                                                          preComputedLandsatCloudScoreOffset = None,
                                                          preComputedLandsatTDOMIRMean = landsat_tdom_stats.select(['.*_mean']),
                                                          preComputedLandsatTDOMIRStdDev = landsat_tdom_stats.select(['.*_stdDev']),
                                                          preComputedSentinel2CloudScoreOffset = None,
                                                          preComputedSentinel2TDOMIRMean = s2_tdom_stats.select(['.*_mean']),
                                                          preComputedSentinel2TDOMIRStdDev = s2_tdom_stats.select(['.*_stdDev'])
                                                           )

Get Processed Landsat and Sentinel2 Scenes: 
Start date: May 31 1984 , End date: May 31 2023
Get Processed Landsat: 
Start date: May 31 1984 , End date: May 31 2023
Defringing L4 and L5
Including All Landsat 7
Setting resample method to  bicubic
Applying Cloud Score
Not computing cloudScore offset
Applying Fmask Cloud Mask
Applying TDOM Shadow Mask
Using pre-computed irMean for TDOM
Using pre-computed irStdDev for TDOM
Applying Fmask Shadow Mask
Get Processed Sentinel2: 
Start date: May 31 1984 , End date: May 31 2023
Using S2 Collection: COPERNICUS/S2_HARMONIZED
Joining pre-computed cloud probabilities from: COPERNICUS/S2_CLOUD_PROBABILITY
Setting resample method to  bicubic
Converting S2 data to daily mosaics
Apply Cloud Probability
Applying TDOM
Using pre-computed irMean for TDOM
Using pre-computed irStdDev for TDOM
pyramiding object: {'.default': 'mean'}
Exporting: Landsat-Sentinel2-Hybrid_TOA_medoid_1984_1984_152_151
<Task EXPORT_IMAGE: Landsat-Sentinel2-Hybrid_TOA_medoid_1984_198

127.0.0.1 - - [03/Aug/2023 21:23:36] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2023 21:23:53] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2023 21:24:07] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2023 21:25:21] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2023 21:25:45] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2023 21:27:23] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2023 21:28:00] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2023 21:28:22] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2023 21:32:16] "GET /geeView/?accessToken=None HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2023 21:32:16] "GET /geeView/css/style.min.css HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2023 21:32:16] "GET /geeView/js/lcms-viewer.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2023 21:32:16] "GET /geeView/js/load.min.js HTTP/1.1" 2

### Manage export tasks
These export tasks can take anywhere from a few hours to a few days to run, depending on the size of your study area. It can be helpful to manage and observe the tasks after you've set them running. 

Below are some code snippets to use to manage and inspect the export tasks.

In [9]:
# If you'd like to track the tasks, use this:
# tml.trackTasks2()

# If you want to cancel all running tasks, you can use this function
# tml.batchCancel()

# If you want to empty the collection of all images
# aml.batchDelete(export_composite_collection, type = 'imageCollection')

print('done')

done


### View composites as they are completed
After your composites have been exported, you can view the composites in your folder. 

In [17]:
# View composites as they are completed
composites = ee.ImageCollection(export_composite_collection)

# If your composites have not yet exported, you can use the following line of code to inspect pre-computed composites instead
# composites = ee.ImageCollection() **update this with a path to our pre-baked composites, wherever they live


### Inspect the outputs

Explore the composites and some attributes that describe the quality of the composites.

In [18]:
# clear the map
Map.clearMap()

# First let's explore the composites and some attributes that can help understand how well the composites turned out
Map.addTimeLapse(composites,getImagesLib.vizParamsFalse10k,'Composites')

# By looking at the Sensor that is used, you can see 
# with the introduction of Landsat 8 in 2013 and then Sentinel 2 in 2016-2017, 
# composites become much better quality
Map.addTimeLapse(composites.select('sensor'),{'min':4,'max':22,'palette':'088,808','classLegendDict':{'Landsat (Landsat 5 = 5, Landsat 7 = 7, etc...)':'088','Sentinel2 (Sentinel 2A = 21, Sentinel 2B = 22, etc....)':'808'}},'Sensor')
Map.addTimeLapse(composites.select('compositeObsCount'),{'min':3,'max':10,'palette':'D0D,0D0'},'Composite Observation Counts')
Map.turnOnInspector()
Map.view()
# Now that we have exported composites, we will use them in the LandTrendr temporal segmentation algorithm

Adding layer: Composites
Adding layer: Sensor
Adding layer: Composite Observation Counts
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:8001/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://3b40cb8a2076f1b6-dot-us-west3.notebooks.googleusercontent.com/proxy/8001/geeView/?accessToken=None


127.0.0.1 - - [09/Aug/2023 22:37:43] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2023 22:37:48] "GET /geeView/images/cumulative_icon.png HTTP/1.1" 200 -


#### Done with Module 2

Now that we have exported composites, we will use them in the LandTrendr temporal segmentation algorithm in module 3. 